In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, cross_val_score

from sklearn.utils import resample
from imblearn.over_sampling import SMOTE

from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, ExtraTreesClassifier

import xgboost as xgb
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from hyperopt.pyll import scope

from sklearn.metrics import roc_auc_score, f1_score

import random
import pickle
import os

%matplotlib inline

/usr/lib/python3/dist-packages/pkg_resources/__init__.py:116: PkgResourcesDeprecationWarning: 0.1.43ubuntu1 is an invalid version and will not be supported in a future release
  warnings.warn(
/usr/lib/python3/dist-packages/pkg_resources/__init__.py:116: PkgResourcesDeprecationWarning: 1.1build1 is an invalid version and will not be supported in a future release
  warnings.warn(


In [2]:
import mlflow

mlflow.set_tracking_uri("sqlite:///mlflow.db")
mlflow.set_experiment("prediction-003")

2022/08/30 05:00:38 INFO mlflow.store.db.utils: Creating initial MLflow database tables...
2022/08/30 05:00:38 INFO mlflow.store.db.utils: Updating database tables
INFO  [alembic.runtime.migration] Context impl SQLiteImpl.
INFO  [alembic.runtime.migration] Will assume non-transactional DDL.
INFO  [alembic.runtime.migration] Running upgrade  -> 451aebb31d03, add metric step
INFO  [alembic.runtime.migration] Running upgrade 451aebb31d03 -> 90e64c465722, migrate user column to tags
INFO  [alembic.runtime.migration] Running upgrade 90e64c465722 -> 181f10493468, allow nulls for metric values
INFO  [alembic.runtime.migration] Running upgrade 181f10493468 -> df50e92ffc5e, Add Experiment Tags Table
INFO  [alembic.runtime.migration] Running upgrade df50e92ffc5e -> 7ac759974ad8, Update run tags with larger limit
INFO  [alembic.runtime.migration] Running upgrade 7ac759974ad8 -> 89d4b8295536, create latest metrics table
INFO  [89d4b8295536_create_latest_metrics_table_py] Migration complete!
INFO  

<Experiment: artifact_location='./mlruns/1', experiment_id='1', lifecycle_stage='active', name='prediction-003', tags={}>

In [ ]:
def process_dataframe(filepath):    
    df = pd.read_csv(filepath)
    
    df.drop(['nativeCountry'], axis=1, inplace=True)
    
    target = 'incomeTarget'

    transformed_target = []

    for _, value in df['incomeTarget'].iteritems():
        if value == ' <=50K':
            transformed_target.append(0)
        else:
            transformed_target.append(1)
    df['incomeTarget'] = transformed_target
    
    y = df[target]
    X = df.drop('incomeTarget', axis=1, inplace=True)
    X = pd.get_dummies(df)

    # Upsample using SMOTE
    sm = SMOTE(random_state=12)
    X_train_sm, y_train_sm = sm.fit_resample(X, y)

    
    # df_new = pd.DataFrame(X_upsampled, columns=X.columns)
    df_new = pd.DataFrame(X_train_sm, columns=X.columns)
    
    return df_new, y_train_sm

In [ ]:
X_train, y_train = process_dataframe('../data/adult-train.csv')
X_val, y_val = process_dataframe('../data/adult-val.csv')

len(X_train), len(y_train), len(X_val), len(y_val)

In [ ]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_train_df = pd.DataFrame(X_train_scaled, columns = X_train.columns)

X_val_scaled = scaler.transform(X_val)
X_val_df = pd.DataFrame(X_val_scaled, columns = X_val.columns)

In [ ]:
dv = DictVectorizer()

train_dicts = X_train.to_dict(orient='records')
X_train = dv.fit_transform(train_dicts)

val_dicts = X_val.to_dict(orient='records')
X_val = dv.transform(val_dicts)

In [ ]:
logreg = LogisticRegression(max_iter=500)
logreg.fit(X_train, y_train)

y_pred = logreg.predict(X_val)

In [ ]:
os.makedirs('models/', exist_ok=True)

with open('models/logreg.bin', 'wb') as f_out:
    pickle.dump((dv, logreg), f_out)

In [ ]:
with mlflow.start_run():
    mlflow.set_tag("developer", "enchristos")
    
    mlflow.log_param("train-data-path", "../data/adult-train.csv")
    mlflow.log_param("valid-data-path", "../data/adult-test.csv")
    
    max_iter = 500
    mlflow.log_param("max_iter", max_iter)
    logreg = LogisticRegression(max_iter=max_iter)
    logreg.fit(X_train, y_train)
    
    y_pred = logreg.predict(X_val)
    auc = roc_auc_score(y_val, y_pred)
    f1 = f1_score(y_val, y_pred)

    mlflow.log_metric("auc", auc)
    mlflow.log_metric("fi_score", f1)
    mlflow.log_metric("loss_metric", 1-f1)
    
    mlflow.log_artifact(local_path="models/logreg.bin", artifact_path="models_pickle")

In [ ]:
with mlflow.start_run():
    mlflow.set_tag("developer", "enchristos")
    
    mlflow.log_param("train-data-path", "../data/adult-train.csv")
    mlflow.log_param("valid-data-path", "../data/adult-test.csv")
    
    learning_rate = 0.2
    mlflow.log_param("learning_rate", learning_rate)
    gbc = GradientBoostingClassifier(learning_rate=learning_rate)
    gbc.fit(X_train, y_train)
    
    y_pred = gbc.predict(X_val)
    
    auc = roc_auc_score(y_val, y_pred)
    f1 = f1_score(y_val, y_pred)
    
    mlflow.log_metric("auc", auc)
    mlflow.log_metric("fi_score", f1)
    mlflow.log_metric("loss_metric", 1-f1)
    
    with open('models/gbc.bin', 'wb') as f_out:
        pickle.dump((dv, gbc), f_out)
    
    mlflow.log_artifact(local_path="models/gbc.bin", artifact_path="models_pickle")

In [ ]:
with mlflow.start_run():
    mlflow.set_tag("developer", "enchristos")
    
    mlflow.log_param("train-data-path", "../data/adult-train.csv")
    mlflow.log_param("valid-data-path", "../data/adult-test.csv")
    
    n_estimators = 100
    mlflow.log_param("n_estimators", n_estimators)
    rfc = RandomForestClassifier(n_estimators=n_estimators)
    rfc.fit(X_train, y_train)
    
    y_pred = rfc.predict(X_val)
    auc = roc_auc_score(y_val, y_pred)
    f1 = f1_score(y_val, y_pred)
    mlflow.log_metric("auc", auc)
    mlflow.log_metric("fi_score", f1)
    mlflow.log_metric("loss_metric", 1-f1)
    
    with open('models/rfc.bin', 'wb') as f_out:
        pickle.dump((dv, gbc), f_out)
    
    mlflow.log_artifact(local_path="models/rfc.bin", artifact_path="models_pickle")

In [ ]:
def objective(params):
    with mlflow.start_run():
        mlflow.set_tag("model", "XGBClassifier")
        mlflow.set_tag("developer", "enchristos")
        mlflow.log_params(params)
        booster = xgb.XGBClassifier(
            n_estimators = params['n_estimators'],
            max_depth = int(params['max_depth']),
            learning_rate = params['learning_rate'],
            gamma = params['gamma'],
            min_child_weight = params['min_child_weight'],
            subsample = params['subsample'],
            colsample_bytree = params['colsample_bytree'],
            random_state = params['seed']
        )
        
        booster.fit(X_train, y_train)
        
        y_pred = booster.predict(X_val)
        auc = roc_auc_score(y_val, y_pred)
        f1 = f1_score(y_val, y_pred)
        mlflow.log_metric("auc", auc)
        mlflow.log_metric("fi_score", f1)
        mlflow.log_metric("loss_metric", 1-f1)

    return {'loss': 1-f1, 'status': STATUS_OK}

In [ ]:
search_space = {
    'max_depth' : scope.int(hp.quniform('max_depth', 4, 10, 0.1)),
    'learning_rate' : hp.quniform('learning_rate', 0.01, 0.5, 0.01),
    'n_estimators' : hp.choice('n_estimators', range(0, 50, 1)),
    'gamma' : hp.quniform('gamma', 0.01, 0.50, 0.01),
    'min_child_weight' : hp.quniform('min_child_weight', 0, 10, 0.1),
    'subsample' : hp.quniform('subsample', 0.1, 1, 0.01),
    'colsample_bytree' : hp.quniform('colsample_bytree', 0.1, 1.0, 0.01),
    'seed': 42
}


best_result = fmin(
    fn=objective,
    space=search_space,
    algo=tpe.suggest,
    max_evals=500,
    trials=Trials()
)

In [ ]:
best_result

In [ ]:
with mlflow.start_run():
    
    best_params = {
         'colsample_bytree': 0.32,
         'gamma': 0.48,
         'learning_rate': 0.32,
         'max_depth': 7.600000000000005,
         'min_child_weight': 4.2,
         'n_estimators': 45,
         'subsample': 0.96,
         'seed': 42
    }

    mlflow.log_params(best_params)

    booster = xgb.XGBClassifier(
            n_estimators = best_params['n_estimators'],
            max_depth = int(best_params['max_depth']),
            learning_rate = best_params['learning_rate'],
            gamma = best_params['gamma'],
            min_child_weight = best_params['min_child_weight'],
            subsample = best_params['subsample'],
            colsample_bytree = best_params['colsample_bytree'],
            random_state = best_params['seed']
        )
        
    booster.fit(X_train, y_train)
        
    y_pred = booster.predict(X_val)
    auc = roc_auc_score(y_val, y_pred)
    f1 = f1_score(y_val, y_pred)
    mlflow.log_metric("auc", auc)
    mlflow.log_metric("fi_score", f1)
    mlflow.log_metric("loss_metric", 1-f1)

    with open("models/preprocessor.b", "wb") as f_out:
        pickle.dump(dv, f_out)
        
    mlflow.log_artifact("models/preprocessor.b", artifact_path="preprocessor")

    mlflow.xgboost.log_model(booster, artifact_path="models_mlflow")

In [ ]:
mlflow.sklearn.autolog()

for model_class in (RandomForestClassifier, GradientBoostingClassifier, ExtraTreesClassifier, xgb.XGBClassifier):
    with mlflow.start_run():
        mlflow.set_tag("developer", "enchristos")
        mlflow.log_param("train-data-path", "../data/adult-train.csv")
        mlflow.log_param("valid-data-path", "../data/adult-test.csv")
        
        mlflow.log_artifact("models/preprocessor.b", artifact_path="preprocessor")
        
        mlmodel = model_class()
        mlmodel.fit(X_train, y_train)
        
        y_pred = mlmodel.predict(X_val)
        auc = roc_auc_score(y_val, y_pred)
        f1 = f1_score(y_val, y_pred)
        mlflow.log_metric("auc", auc)
        mlflow.log_metric("fi_score", f1)
        mlflow.log_metric("loss_metric", 1-f1)

In [3]:
mlflow.get_artifact_uri()

'./mlruns/1/f676112f1f02450db694fc733ec3cbc9/artifacts'